In [1]:
import os
import glob
import pandas as pd
import numpy as np

In [2]:
metrics = ['seed', 't_training_avg', 't_training_std',
            'RMSEValidation','EnergyScoreValidation', 'CRPSValidation',
            'Gaussian NLLValidation', 'CoverageValidation']

path = "../results/KS/hparams/beta/mixednormal/" #"../results/Burgers/hparams/mvnormal/"
file_list = ([f for f in glob.iglob(path + "**", recursive = True) if os.path.isfile(f) and f.endswith("test.csv")])
ids = ["beta_endpoints"]
for id in ids:
    metrics.append(id)
factor = 10

In [3]:
results = pd.DataFrame()
for file in file_list:
    if os.path.exists(file):
        results_df = pd.read_csv(file, index_col=0)
        results = pd.concat([results, results_df], axis = 1)
rows = metrics.copy()
results = results.loc[rows]
metrics.pop()
results.loc[metrics] = results.loc[metrics].astype("float32")
results = results.transpose()
results.index = results[id]
results.drop(id, axis = 1, inplace = True)
results.sort_values(ids, inplace = True)
results.insert(0, "beta",[0.2,0.2,0.35,0.35])
results["beta"] = [0.2,0.2,0.35,0.35]

In [4]:
results

,beta,seed,t_training_avg,t_training_std,RMSEValidation,EnergyScoreValidation,CRPSValidation,Gaussian NLLValidation,CoverageValidation
beta_endpoints,,,,,,,,,
"(0.001, 0.2)",0.20,12345.0,10.515783,0.088675,0.004909,0.065604,0.00257,-3.807157,0.999566
"[0.001, 0.2]",0.20,1234.0,10.493962,0.083198,0.004583,0.06336,0.002594,-3.827182,0.999722
"[0.001, 0.35]",0.35,1234.0,10.599971,0.226976,0.006264,0.083255,0.003415,-3.560698,0.999596
"[0.001, 0.35]",0.35,12345.0,10.515601,0.087133,0.004858,0.063456,0.002589,-3.803216,0.999431


In [5]:
results = results.groupby(by = ["beta"]).mean().drop(["seed", "t_training_avg", "t_training_std"], axis = 1)
results.iloc[:,0:3] *= factor
results

,RMSEValidation,EnergyScoreValidation,CRPSValidation,Gaussian NLLValidation,CoverageValidation
beta,,,,,
0.20,0.047458,0.644821,0.025816,-3.81717,0.999644
0.35,0.055613,0.733557,0.030022,-3.681957,0.999514


In [6]:
latex_table = results.to_latex(escape=False, float_format="%.4f")
print(latex_table)

\begin{tabular}{llllll}
\toprule
 & RMSEValidation & EnergyScoreValidation & CRPSValidation & Gaussian NLLValidation & CoverageValidation \\
beta &  &  &  &  &  \\
\midrule
0.200000 & 0.0475 & 0.6448 & 0.0258 & -3.8172 & 0.9996 \\
0.350000 & 0.0556 & 0.7336 & 0.0300 & -3.6820 & 0.9995 \\
\bottomrule
\end{tabular}

